In [33]:
# feature grabbing
import os
import numpy as np

def feature_from_file(file_path, feature_type="head", byte_num=512): # will add more feature_type later
    """Retreives features from a file.
  
    Parameters:
    feature_type (str): "head" to get bytes from head of the file.
    byte_num (int): Number of bytes to grab.
    file_path (str): File path of file to get features from.
    
    Returns:
    List of bytes from file_path. 
    """
    with open(file_path, 'rb') as f:
        byte = f.read(1)
        index = 1
        features = []
    
        while byte and index <= byte_num:
            features.append(byte)
            index += 1
            byte = f.read(1)
        
        if len(features) < byte_num:
            features.extend([b'' for i in range(byte_num - len(features))])

        assert len(features) == byte_num
        return features

def feature_from_dir(dir_path, feature_type="head", byte_num=512):
    """Takes a directory and grabs features from each file.
    
    Parameters:
    dir_path (str): Path of directory to take features from.
    feature_type (str): Type of features to get.
    byte_num (str): Number of features to take
    
    Return:
    features (list): List containing a list of byte_num bytes from each fie in dir_path.
    """
    features = []
    for (dirpath, dirnames, filenames) in os.walk(dir_path):
        for filename in filenames:
            features.append(feature_from_file(os.path.join(dirpath, filename), feature_type, byte_num))
    
    return features

def translate_bytes(dir_features):
    """Translates bytes into integers.
    
    Parameter:
    dir_features (list): List containing lists of bytes.
    
    Return:
    translated_features (numpy array): dir_features with bytes translated to integers.
    """
    translated_features = np.zeros((len(dir_features), len(dir_features[0])))
    
    for idx, file_features in enumerate(dir_features):
        translated_features[idx] = np.array([int.from_bytes(c, byteorder="big") for c in file_features])
    
    return translated_features

0.0


In [36]:
from sklearn.model_selection import train_test_split

test_set_dir = ''
x = translate_bytes(feature_from_dir(test_set_dir))

x_train, x_test, _, _ = train_test_split(x, x)

IndexError: list index out of range

In [37]:
from keras.models import Model
from keras.layers import Input, Dense

input_size = len(x_train[0])

input_layer = Input((input_size,))
encoded = Dense(128, activation='relu')(input_layer)
encoded = Dense(64, activation='relu')(encoded)
encoded = Dense(32, activation='relu')(encoded)

decoded = Dense(64, activation='relu')(encoded)
decoded = Dense(128, activation='relu')(decoded)
decoded = Dense(input_size, activation='sigmoid')(decoded)

autoencoder = Model(inputs=input_layer, outputs=decoded)
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy', metrics=['accuracy'])
autoencoder.fit(x_train, x_train,
               epochs=100,
               batch_size = 256,
               shuffle=True,
               validation_data=(x_test, x_test))

encoder = Model(input_img, encoded)

Using TensorFlow backend.


ModuleNotFoundError: No module named 'tensorflow'